# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [40]:
# checking your current working directory
print("Current directory:")
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print("List of filepaths:")
    print(file_path_list)

Current directory:
/home/workspace
List of filepaths:
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/wo

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("Total number of rows:")
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows:
8056


In [42]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Number of rows in event_datafile_new.csv:")
    print(sum(1 for line in f))

Number of rows in event_datafile_new.csv:
6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [43]:
# Make a connection to a Cassandra instance from your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, a session is needed
session = cluster.connect()

#### Create Keyspace

In [44]:
# This will create a keyspace called udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [45]:
# Setting keyspace to the keyspace 'udacity' specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. List the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. List only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. List every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Create table and insert rows into created table for query 1: List the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [46]:
# Create table 'song_length_music_library' for query 1. Primary key is sessionId and itemInSession to make it unique.
query = "CREATE TABLE IF NOT EXISTS song_length_music_library "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [47]:
# Read CSV file then insert rows from file into table 'song_length_music_library' for query 1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert sessionId, itemInSession, artist, song, length into 'song_length_music_library' table
        query = "INSERT INTO song_length_music_library (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement for query 1 to confirm data is inserted into new table

In [48]:
# Execute select statement from table 'song_length_music_library' for query 1 
query = "select artist, song, length from song_length_music_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = {}
list_rows = []    
for row in rows:
    # Store each row into a 'data' dictionary
    data = {'artist':row.artist,'song':row.song, 'length':row.length}
    # Append each 'data' dictionary into a list called 'list_rows'
    list_rows.append(data)
    
# Create a Datafame with 'list_rows'
df = pd.DataFrame(list_rows)
# Display the DataFrame
df

,artist,length,song
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


## Create table and insert rows into created table for query 2: List only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [49]:
# Create table 'song_user_music_library' for query 2. The userid and sessionId are the partition keys.
# ItemInSesison is a clustering column since query is ordered by this column.
query = "CREATE TABLE IF NOT EXISTS song_user_music_library "
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [50]:
# Read CSV file then insert rows from file into table 'song_user_music_library' for query 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert userid, sessionId, itemInSession, artist, song, firstName, lastName into 'song_user_music_library' table
        query = "INSERT INTO song_user_music_library (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT statement for query 2 to confirm data is inserted into new table

In [51]:
# Execute select statement from table 'song_user_music_library' for query 2
query = "select itemInSession, artist, song, firstName, lastName from song_user_music_library WHERE userid = 10 and sessionId = 182 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = {}
list_rows = []
for row in rows:
    # Store each row into a 'data' dictionary
    data = {'item in session':row.iteminsession, 'artist':row.artist,'song':row.song, 'first name':row.firstname, 'last name':row.lastname}
    list_rows.append(data)

# Create to a Datafame with 'list_rows'
df = pd.DataFrame(list_rows)    
# Display the DataFrame
df



,artist,first name,item in session,last name,song
0,Down To The Bone,Sylvie,0,Cruz,Keep On Keepin' On
1,Three Drives,Sylvie,1,Cruz,Greece 2000
2,Sebastien Tellier,Sylvie,2,Cruz,Kilometer
3,Lonnie Gordon,Sylvie,3,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


## Create table and insert rows into created table for query 3: List every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [52]:
# Create table 'username_music_library for query 3. Primary key is song and userid for uniqueness.
query = "CREATE TABLE IF NOT EXISTS username_music_library "
query = query + "(song text, userid int, firstName text, lastName text, PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [53]:
# Read CSV file then insert rows from file into table 'username_music_library' for query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert song, userid, firstName, lastName into 'username_music_library' table
        query = "INSERT INTO username_music_library (song, userid, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT statement for query 3 to confirm data is inserted into new table

In [54]:
# Execute select statement from table 'username_music_library' for query 3
query = "select firstName, lastName from username_music_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = {}
list_rows = []    
for row in rows:
    # Store each row into a 'data' dictionary
    data = {'first name':row.firstname,'last name':row.lastname}
    # Append each 'data' dictionary into a list called 'list_rows'
    list_rows.append(data)
    
# Create a Datafame with 'list_rows'
df = pd.DataFrame(list_rows)
# Display the DataFrame
df

,first name,last name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [55]:
# Drop the 3 created tables above before closing out the session
query = "DROP TABLE IF EXISTS song_length_music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_user_music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS username_music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [56]:
session.shutdown()
cluster.shutdown()